<a href="https://colab.research.google.com/github/sanujsriv/_topic_models/blob/FoTo/May2022_FoTo_query_selection_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### just run these

In [ ]:
import os
import pandas as pd
import numpy as np

#@title Imports, wordnet,punkt
import nltk
import glob
# nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import re
import string
from time import time
import numpy as np
import collections
from sklearn.feature_extraction.text import CountVectorizer
from numpy import random
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#@title function : load / save pickle_obj

### pickle

import bz2
import pickle
!pip install pickle5
import pickle5
import _pickle as cPickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def load_obj_pkl5(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle5.load(f)

def compressed_pickle(data,title):
  with bz2.BZ2File(title + '.pbz2', 'w') as f:
    cPickle.dump(data, f)

def decompress_pickle(file):
 data = bz2.BZ2File(file+'.pbz2', 'rb')
 data = cPickle.load(data)
 return data

In [ ]:
if 'grad16' or 'student' in os.getcwd(): local=True
else: local = False
if local:
  paper='emnlp2022'
  home_dir = '/home/grad16/sakumar/'+paper+'/dataset'
elif not local:
  home_dir = ''
os.chdir(home_dir)

###load word2vec (takes time...!)

In [ ]:
from gensim import models
if local==True:
  if not os.path.isfile("/home/student_no_backup/sakumar/miniconda3/bin/GoogleNews-vectors-negative300.bin"):
    !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    !gunzip GoogleNews-vectors-negative300.bin.gz
  word2vec_model = models.KeyedVectors.load_word2vec_format('/home/student_no_backup/sakumar/miniconda3/bin/GoogleNews-vectors-negative300.bin', binary=True)

if local==False:
  # https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
  !gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM
  ## OR (not working)
  # !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O GoogleNews-vectors-negative300.bin.gz && rm -rf /tmp/cookies.txt
  ## OR (not working)
  # !wget -N -c "https://raw.githubusercontent.com/mmihaltz/word2vec-GoogleNews-vectors/master/GoogleNews-vectors-negative300.bin.gz"
  !gunzip GoogleNews-vectors-negative300.bin.gz
  word2vec_model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

KeyboardInterrupt: ignored

## Preprocessing

### Preprocessing helper functions

In [ ]:
stem = PorterStemmer()
wnl = WordNetLemmatizer()
stopwords = ['http','la','wa','will','fa','ha','pa','co','v','said','cant','better','well','going','will','would','know','dont','get','like','think','im',"also","said","a", "able", "about", "above", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "after", "afterwards", "again", "against", "ah", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "auth", "available", "away", "awfully", "b", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "between", "beyond", "biol", "both", "brief", "briefly", "but", "by", "c", "ca", "came", "can", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "could", "couldnt", "d", "date", "did", "didn't", "different", "do", "does", "doesn't", "doing", "done", "don't", "down", "downwards", "due", "during", "e", "each", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "for", "former", "formerly", "forth", "found", "four", "from", "further", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "had", "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "hed", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hers", "herself", "hes", "hi", "hid", "him", "himself", "his", "hither", "home", "how", "howbeit", "however", "hundred", "i", "id", "ie", "if", "i'll", "im", "immediate", "immediately", "importance", "important", "in", "inc", "indeed", "index", "information", "instead", "into", "invention", "inward", "is", "isn't", "it", "itd", "it'll", "its", "itself", "i've", "j", "just", "k", "keep  keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "more", "moreover", "most", "mostly", "mr", "mrs", "much", "mug", "must", "my", "myself", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "now", "nowhere", "o", "obtain", "obtained", "obviously", "of", "off", "often", "oh", "ok", "okay", "old", "omitted", "on", "once", "one", "ones", "only", "onto", "or", "ord", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "re", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "s", "said", "same", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "she", "shed", "she'll", "shes", "should", "shouldn't", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure    t", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'll", "theyre", "they've", "think", "this", "those", "thou", "though", "thoughh", "thousand", "throug", "through", "throughout", "thru", "thus", "til", "tip", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "very", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "was", "wasnt", "way", "we", "wed", "welcome", "we'll", "went", "were", "werent", "we've", "what", "whatever", "what'll", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "whose", "why", "widely", "willing", "wish", "with", "within", "without", "wont", "words", "world", "would", "wouldnt", "www", "x", "y", "yes", "yet", "you", "youd", "you'll", "your", "youre", "yours", "yourself", "yourselves", "you've", "z", "zero"]

def preprocessing(doc,word2vec_model,my_punctuation,min_doc_length,min_word_length=0):
    doc = doc.lower()
    doc = doc.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))  # string.punctuation[6] = ' won't --> won , don't --> don
    doc = word_tokenize(doc)
    doc = filter(lambda x: x not in my_punctuation, doc)
    doc = filter(lambda x:not x.isdigit(), doc)
    doc = [wnl.lemmatize(w.lower()) for w in doc]
    #doc = [stem.stem(w) for w in doc]
    doc = filter(lambda x:x not in stopwords, doc)
    doc = filter(lambda x: x in word2vec_model.vocab or x in ".",doc)
    doc = list(doc)
    doc = [e for e in doc if len(e)>=min_word_length]
    if len(doc)>=min_doc_length:
      doc = ' '.join(e for e in doc)
    else:
      doc = ''
    return doc

In [ ]:
def vocab_filtered_data(doc,vocab):
  doc = word_tokenize(doc)
  doc = filter(lambda x: x in vocab, doc)
  doc = ' '.join(e for e in doc)
  return doc

def flatten(t):
    return [item for sublist in t for item in sublist]


def docs_labels_preprocessing(data_preprocessed,data_preprocessed_labels,word2vec_model,min_doc_len,min_word_length,max_features):
  embeddings = {}

  vectorizer = CountVectorizer(max_features=max_features,dtype=np.uint8)
  train_vec = vectorizer.fit_transform(data_preprocessed).toarray()
  vocab = vectorizer.vocabulary_

  nonzeros_indexes = np.where(train_vec.any(1))[0]
  data_preprocessed = [data_preprocessed[i] for i in nonzeros_indexes]
  data_preprocessed_labels = [data_preprocessed_labels[i] for i in nonzeros_indexes]

  for i in range(len(data_preprocessed)):
    data_preprocessed[i] = vocab_filtered_data(data_preprocessed[i],vocab)

  data_preprocessed_f = [data_preprocessed[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]
  data_preprocessed_labels_f = [data_preprocessed_labels[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]

  vectorizer_f = CountVectorizer(dtype=np.uint8)
  vectorizer_f.fit_transform(data_preprocessed_f)
  vocab_f = vectorizer.vocabulary_

  for f in vocab_f:
    embeddings[f] = word2vec_model[f]
  return data_preprocessed_f,data_preprocessed_labels_f,embeddings,vocab_f


def docs_labels_preprocessing_sent(docs,labels,word2vec_model,min_doc_len,min_word_length,max_features,split_to_sentences):
  data_preprocessed = []
  data_preprocessed_labels = []
  embeddings = {}
  data_preprocessed_f = []
  data_preprocessed_labels_f = []
  vocab_f = {}

  for i in range(len(docs)):

    doc = preprocessing(docs[i],word2vec_model,string.punctuation,min_doc_len,min_word_len)
    if len(doc)!=0:
      data_preprocessed.append(doc)
      data_preprocessed_labels.append(labels[i])

  if len(data_preprocessed)!=0:
    vectorizer = CountVectorizer(max_features=max_features,dtype=np.float32)
    train_vec = vectorizer.fit_transform(data_preprocessed).toarray()
    vocab = vectorizer.vocabulary_

    nonzeros_indexes = np.where(train_vec.any(1))[0]
    data_preprocessed = [data_preprocessed[i] for i in nonzeros_indexes]
    data_preprocessed_labels = [data_preprocessed_labels[i] for i in nonzeros_indexes]

    for i in range(len(data_preprocessed)):
      data_preprocessed[i] = vocab_filtered_data(data_preprocessed[i],vocab)


    if not split_to_sentences:
      data_preprocessed_f = [data_preprocessed[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]
      data_preprocessed_labels_f = [data_preprocessed_labels[i] for i in range(len(data_preprocessed)) if len(data_preprocessed[i].split(' '))>=min_doc_len]


    if split_to_sentences:
      data_preprocessed_f = [data_preprocessed[i] for i in range(len(data_preprocessed)) if len(data_preprocessed)>1 or len(data_preprocessed[i].split(' '))>=min_doc_len]
      data_preprocessed_labels_f = [data_preprocessed_labels[i] for i in range(len(data_preprocessed)) if len(data_preprocessed)>1 or len(data_preprocessed[i].split(' '))>=min_doc_len]


    if len(data_preprocessed_f)!=0:
      vectorizer_f = CountVectorizer(dtype=np.uint8)
      vectorizer_f.fit_transform(data_preprocessed_f)
      vocab_f = vectorizer.vocabulary_

    for f in vocab:
      embeddings[f] = word2vec_model[f]
  return data_preprocessed_f,data_preprocessed_labels_f,embeddings,vocab_f

def get_corpus_len_info(data_preprocessed,name):
  len_docs = [len(d.split(" ")) for d in data_preprocessed]
  print('name :',name)
  print('min, mean, max docs len  : ',str(np.min(len_docs))+', '+str(np.mean(len_docs).round(2))+', '+str(np.max(len_docs))+'\n\n')

### bbc

In [ ]:
def get_bbc():
  dtype='short'
  os.system('wget -N https://www.dropbox.com/s/vunli21d312x55g/bbc.zip')
  os.system('wget -N https://www.dropbox.com/s/h6y9zfdb76gl4uz/bbc-fulltext.zip')
  os.system('unzip bbc.zip')
  os.system('unzip bbc-fulltext.zip')

  # BBC Docs -
  corpus = []
  subfolders = [f.path for f in os.scandir(os.getcwd()+'/bbc') if f.is_dir()]
  subfolders = sorted(subfolders)
  for s in subfolders:
    files_list = sorted(glob.glob(s+"/*.txt"))
    for file in files_list:
      with open(file, "rb") as f:
        content = f.readlines()
        if dtype == 'short':
          content = [content[0],content[2]] # headlines and abstracts
        content = [x.strip().lower().decode('ISO-8859-1') for x in content]
        corpus.append(' '.join(content).strip())

  # BBC_labels -
  with open("bbc.classes", "r") as f:
    content = f.readlines()
    content = [x.strip()[::-1][0] for x in content]
    labels = content[4:]
    label_dict = {'0':'business','1':'entertainment','2':'politics','3':'sport','4':'tech'}
  for l in range(len(labels)):
    labels[l] = label_dict[labels[l]]
  return corpus,labels

### searchsnippet


In [ ]:
def get_searchsnippet():
  os.system('wget http://jwebpro.sourceforge.net/data-web-snippets.tar.gz')
  os.system('tar -xvzf data-web-snippets.tar.gz')
  corpus = []
  labels=[]
  with open("./data-web-snippets/train.txt", "r") as f:
    content = f.readlines()
    content = [x.split('\n')[0] for x in content]
    labels_train = [x.split(' ')[-1] for x in content]
    content = [' '.join(x.split(' ')[:-1]) for x in content]
    corpus.extend(content)
    labels.extend(labels_train)

  with open("./data-web-snippets/test.txt", "r") as f:
    content = f.readlines()
    content = [x.split('\n')[0] for x in content]
    labels_test = [x.split(' ')[-1] for x in content]
    content = [' '.join(x.split(' ')[:-1]) for x in content]
    corpus.extend(content)
    labels.extend(labels_test)
  return corpus,labels

### yahooanswers

In [ ]:
import csv
def get_yahooanswers():
  dtype = 'short'
  gdrive_fileid = """0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU"""
  gdrive_filename = """yahoo_answers_csv.tar.gz"""

  dir= home_dir+'/yahoo_answers_csv'
  if not os.path.exists(dir) and not os.path.isfile(gdrive_filename):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0Bz8a_Dbh9Qhbd2JNdDBsQUdocVU" -O yahoo_answers_csv.tar.gz && rm -rf /tmp/cookies.txt
    os.system("tar -xvzf yahoo_answers_csv.tar.gz")
  # os.makedirs(home_dir+dir,exist_ok=True)
  os.chdir(dir)
  dict_labels = {'1':'Society & Culture', '2': 'Science & Mathematics',
                '3': 'Health', '4': 'Education & Reference', '5' : 'Computers & Internet',
                '6': 'Sports', '7': 'Business & Finance', '8': 'Entertainment & Music',
                '9': 'Family & Relationships' , '10':'Politics & Government'}

  labels = []
  corpus = []
  files = ['train.csv','test.csv']
  # files = ['train.csv']
  count = 0
  for f in files:
    with open(f, newline='') as csvfile:
        reader = csv.reader(csvfile)
  #       # next(reader)
        if dtype == 'short':
          for row in reader:
            count = count+1
            labels.append(dict_labels[row[0]])
            # corpus.append(' '.join(t for t in row[1:]))
            corpus.append(' '.join(t for t in row[1:3])) # SHORT
        elif dtype == "full":
          for row in reader:
            count = count+1
            labels.append(dict_labels[row[0]])
            corpus.append(' '.join(t for t in row[1:]))
  return corpus,labels

### nfcorpus

https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/

In [ ]:
## nfcorpus
import pandas as pd
import glob

def get_nfcorpus():
  !wget -N -c https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz
  !tar -xvzf nfcorpus.tar.gz

  os.chdir(home_dir+"/nfcorpus/raw")

  with open('doc_dump.txt', "r") as f: # titles only
    content = f.readlines()
    doc_dump_ids = [c.split('\t')[0] for c in content]
    doc_dump_titles = [c.split('\t')[2].split('- PubMed')[0].strip() for c in content]
  doc_dump_dict = dict(zip(doc_dump_ids, doc_dump_titles))
  get_corpus_len_info(doc_dump_titles,'doc_dump.txt')

  os.chdir(home_dir+"/nfcorpus")
  queries_file_list = ['dev.titles.queries','train.titles.queries','test.titles.queries']

  query_docs_ids, queries_docs = [],[]
  for file in queries_file_list:
      with open(file, "r") as f:
        content = f.readlines()
        query_docs_ids.extend([c.split('\t')[0] for c in content])
        queries_docs.extend([c.split('\t')[1].strip() for c in content])
  query_dict = dict(zip(query_docs_ids, queries_docs))

  qrels_file_list = ['dev.2-1-0.qrel','train.2-1-0.qrel','test.2-1-0.qrel']

  path = home_dir+"/nfcorpus" # use your path
  all_files = glob.glob(path + "/*.2-1-0.qrel")

  li = []

  for filename in all_files:
      df = pd.read_csv(filename,sep ='\t',header=None)
      li.append(df)

  frame = pd.concat(li, axis=0, ignore_index=True)
  frame.columns = ['QUERY_ID', '0', 'DOC_ID', 'RELEVANCE_LEVEL']
  frame = frame.drop(['0'],axis=1)
  frame['QUERY'] = frame['QUERY_ID'].apply(lambda x:  query_dict[x])
  frame['DOC_TITLE'] = frame['DOC_ID'].apply(lambda x:  doc_dump_dict[x])

  frame.replace('', np.nan, inplace=True)
  frame.isnull().sum()
  frame = frame.dropna().reset_index(drop=True)
  return frame

In [ ]:
# docs_idx = frame[['DOC_TITLE_PREPROCESSED']].drop_duplicates().reset_index(drop=True).index.tolist()
# docs_label_df = frame.iloc[docs_idx]
# docs = docs_label_df['DOC_TITLE_PREPROCESSED'].values
# labels = docs_label_df['DOC_ID'].values#.apply(lambda x: x.split('-')[0]).values

# min_doc_len,min_word_len = 3,3
# # preprocessing('Hello !! bro> aa',word2vec_model,string.punctuation,min_doc_len,min_word_len)
# frame['DOC_TITLE_PREPROCESSED'] = frame['DOC_TITLE'].apply(lambda x: preprocessing(x,word2vec_model,string.punctuation,min_doc_len,min_word_len))

# docs_label_df[docs_label_df['RELEVANCE_LEVEL']==2][['QUERY','DOC_ID','RELEVANCE_LEVEL']].values
# docs_label_df[docs_label_df['QUERY']=='organic milk and prostate cancer'][['RELEVANCE_LEVEL','DOC_TITLE_PREPROCESSED']].values
# foods for macular degeneration , how to boost the benefits of exercise, organic milk and prostate cancer

### twitter

In [ ]:
def get_opinions_twitter():
  ## twitter
  !wget -N -c https://www.dropbox.com/s/obu9dve4ul5c0up/Corpus.tar
  !tar -xvzf Corpus.tar
  os.chdir(home_dir+"/Corpus")

  with open('DATA', "r") as f: # titles only
    content = f.readlines()
    queries = [x.split('#')[1].strip() for x in content if "#" in x]
    tweet_ids , relevance = map(list,zip(*[x.split('\t') for x in content if "#" not in x]))
    relevance = [x.strip() for x in relevance]

  os.chdir(home_dir+'/etc')
  tweet_text = decompress_pickle('tweet_text_5000')
  os.chdir(home_dir)
  repeated_queries = np.repeat(queries,100)
  tweets= []
  f_queries = []
  f_relevance = []
  f_tweet_ids = []
  for i in range(len(tweet_text)):
    if len(tweet_text[i])==0:
      pass
    else:
      tweets.append(tweet_text[i])
      f_queries.append(repeated_queries[i])
      f_relevance.append(relevance[i])
      f_tweet_ids.append(tweet_ids[i])
  labels = len(tweets)*['tweets']
  df = pd.DataFrame()
  df['tweet_ids'] = f_tweet_ids
  df['tweets'] = tweets
  df['labels'] = labels
  df['queries'] = f_queries
  df['relevance'] = f_relevance
  df.replace('', np.nan, inplace=True)
  df.isnull().sum()
  df = df.dropna().reset_index(drop=True)
  get_corpus_len_info(tweets,'tweets_opinion_retreival')
  return df

In [ ]:
"""
API Key
HQBwNF26a7scxTkRoAsCdTr4E


API Key Secret
eEQ5nkwl1C7qTCg5wO7TesPeWVAOpJBGMPtXQZwE8SArbbNLTq

Bearer Token
AAAAAAAAAAAAAAAAAAAAANyvcQEAAAAAnhZigGSgKGTrht0vpSAyit3mbyE%3DCOXrscMFyXzNDCDYg9m0SkpSsC3KUnn1KUGcTUC5c5YL7bTnEQ

Access Token
1103974251491663873-NOZFV4f0MoAWkhtaG8vVtoORfApe34

Access Token Secret
ATFYBvajprMvhyKRntNpLQEe6qKjjmIODaMDySdZqFpHB
"""

In [ ]:
# !pip install tweepy
############## 1 #################
# def lookup_tweets(tweet_IDs, api):
#     full_tweets = []
#     tweet_count = len(tweet_IDs)
#     try:
#         for i in range((tweet_count / 100) + 1):
#             # Catch the last group if it is less than 100 tweets
#             end_loc = min((i + 1) * 100, tweet_count)
#             full_tweets.extend(
#                 api.statuses_lookup(id=tweet_IDs[i * 100:end_loc])
#             )
#         return full_tweets
#     except:
#         print('Something went wrong, quitting...')


# consumer_key = "HQBwNF26a7scxTkRoAsCdTr4E"
# consumer_secret = "eEQ5nkwl1C7qTCg5wO7TesPeWVAOpJBGMPtXQZwE8SArbbNLTq"
# access_token = "1103974251491663873-NOZFV4f0MoAWkhtaG8vVtoORfApe34"
# access_token_secret = "ATFYBvajprMvhyKRntNpLQEe6qKjjmIODaMDySdZqFpHB"

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True)
# results = lookup_tweets(tweet_ids, api)

# # for tweet in results:
# #     if tweet:
# #         print(tweet.text)

########## 2 ###########
# import time
# import tweepy
# tweet_text=[]
# consumer_key = "HQBwNF26a7scxTkRoAsCdTr4E"
# consumer_secret = "eEQ5nkwl1C7qTCg5wO7TesPeWVAOpJBGMPtXQZwE8SArbbNLTq"
# access_token = "1103974251491663873-NOZFV4f0MoAWkhtaG8vVtoORfApe34"
# access_token_secret = "ATFYBvajprMvhyKRntNpLQEe6qKjjmIODaMDySdZqFpHB"

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

# # calling the api
# api = tweepy.API(auth, wait_on_rate_limit=True)
# for tweet_id in tweet_ids:
#   try:
#     time.sleep(1)
#     # fetching the status
#     status = api.get_status(tweet_id)

#     # fetching the text attribute
#     text = status.text
#   except:
#     text=''
#   tweet_text.append(text)

############# 3 ################
# import tweepy
# CONSUMER_KEY = "HQBwNF26a7scxTkRoAsCdTr4E"
# CONSUMER_SECRET = "eEQ5nkwl1C7qTCg5wO7TesPeWVAOpJBGMPtXQZwE8SArbbNLTq"
# OAUTH_TOKEN = "1103974251491663873-NOZFV4f0MoAWkhtaG8vVtoORfApe34"
# OAUTH_TOKEN_SECRET = "ATFYBvajprMvhyKRntNpLQEe6qKjjmIODaMDySdZqFpHB"

# auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
# api = tweepy.API(auth)
# tweet_text=[]
# for tweet_id in tweet_ids:
#   try:
#     tweet = api.get_status(tweet_id)
#     tweet_text.append(tweet.text)
#   except:
#     tweet_text.append("")
# # compressed_pickle(tweet_text,'tweet_text')
# compressed_pickle(tweet_text,'tweet_text_'+str(len(tweet_text)))


In [ ]:
# failed_tweet_ids = []
# for t in range(len(tweet_text)):
#   if (len(tweet_text[t])==0):
#     failed_tweet_ids.append(tweet_ids[t])
# compressed_pickle(failed_tweet_ids,'failed_tweet_ids')

In [ ]:
# from collections import Counter
# Counter(df['queries'].values)
# df[df['queries']=='iran nuclear'][['relevance','tweets_preprocessed']].values

# # Counter(df['queries'].values)
# Counter(df[df['queries']=='big bang']['relevance'].values)

## Data Generation begins here..

In [ ]:
data_to_get = 'opinions_twitter' # bbc,searchsnippet,yahooanswers,nfcorpus, opinions_twitter
dtype = 'short'

In [ ]:
os.chdir(home_dir)
%pwd

'/home/grad16/sakumar/emnlp2022/dataset'

In [ ]:
if data_to_get == 'bbc':
  docs,labels = get_bbc()
elif data_to_get == 'searchsnippet':
  docs,labels = get_searchsnippet()
elif data_to_get == 'yahooanswers':
  docs,labels = get_yahooanswers()
elif data_to_get == 'nfcorpus':
  frame = get_nfcorpus()
  docs = frame['DOC_TITLE'].values
  doc_id = frame['DOC_ID'].apply(lambda x: x.split('-')[1]).values
  labels = frame['DOC_ID'].apply(lambda x: x.split('-')[0]).values
  queries = frame['QUERY'].values
  relevance = frame['RELEVANCE_LEVEL'].values

elif data_to_get == 'opinions_twitter':
  frame = get_opinions_twitter()
  docs = frame['tweets'].values
  labels = frame['labels'].values
  queries = frame['queries'].values
  relevance = frame['relevance'].values

--2022-06-17 15:30:40--  https://www.dropbox.com/s/obu9dve4ul5c0up/Corpus.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/obu9dve4ul5c0up/Corpus.tar [following]
--2022-06-17 15:30:40--  https://www.dropbox.com/s/raw/obu9dve4ul5c0up/Corpus.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc616656e52d68dcc7d1eaaa9f8b.dl.dropboxusercontent.com/cd/0/inline/BnYTpFMd6Uri7bT5F7B-znTiC4Idx1fHbVGk85TMUZ1mLmg8_X7N-7nRGLc1fv2kXc0Hrd3Hx2DwlWIVjIc4UaHNC7ihJ4Eoy-mRcul90k87s6UvWLPVC_P4ElgUu4xU4St6Z4FDOhOyc_EQO0M6UkKs3Bg-slSMOTyN3P55FqkUPG2kOLWUNCGck0_Umf-saUY/file# [following]
--2022-06-17 15:30:40--  https://uc616656e52d68dcc7d1eaaa9f8b.dl.dropboxusercontent.com/cd/0/inline/BnYTpFMd6Uri7bT5F7B-znTiC4Idx1fHbVGk85TMUZ1mLmg8_X7N-

In [ ]:
df = pd.DataFrame()
df['docs'] = docs
df['labels'] = labels

if data_to_get == 'yahooanswers':
  sample_size = 5000
  seed = 2022
  df = df.groupby('labels').apply(lambda x: x.sample(sample_size,random_state=seed))

### save query too
if data_to_get == 'opinions_twitter' or data_to_get == 'nfcorpus':
  df['queries'] = queries
  df['relevance'] = relevance
  if data_to_get == 'nfcorpus':
    df['doc_id'] = doc_id

min_doc_len,min_word_len = 3,3
df['docs_preprocessed'] = df['docs'].apply(lambda x: preprocessing(x,word2vec_model,string.punctuation,min_doc_len,min_word_len))
df = df.drop_duplicates(subset=['docs_preprocessed'], keep='first').reset_index(drop=True)
data_preprocessed = df['docs_preprocessed'].values
data_preprocessed_labels = df['labels'].values
assert len(data_preprocessed) == len(data_preprocessed_labels)
perc_vocab= 0.7
vectorizer = CountVectorizer(dtype=np.uint8)
train_vec = vectorizer.fit_transform(data_preprocessed).toarray()
all_vocab = vectorizer.vocabulary_

if data_to_get == 'bbc':
  max_features = 2000
elif data_to_get == 'nfcorpus':
  max_features = 2000
elif data_to_get == 'opinions_twitter':
  max_features = 2000
elif data_to_get == 'searchsnippet':
  max_features = 3000
elif data_to_get == 'yahooanswers':
  max_features = 4000

In [ ]:
import gensim

# max_features=None
os.chdir(home_dir+"/content")
os.makedirs(home_dir+'/content/data_'+data_to_get+"/"+dtype,exist_ok=True)
os.chdir(home_dir+'/content/data_'+data_to_get+"/"+dtype)

with open(data_to_get+'_'+dtype+".txt", "a") as f:
  f.write(data_to_get+" - "+dtype)
  f.write("\n\n")
  f.write('**'*50)
  f.write("\n\n")
  data_preprocessed,data_preprocessed_labels,embeddings,vocab = docs_labels_preprocessing(data_preprocessed,data_preprocessed_labels,word2vec_model,min_doc_len,min_word_len,max_features)
  f.write('\n\nlen of - \n  data_preprocessed: '+str(len(data_preprocessed))+'\n  data_preprocessed_labels: '+str(len(data_preprocessed_labels))+'\n  vocab:  '+str(len(vocab))+'\n  embeddings : '+str(len(embeddings))+'\n')
  # f.write(str(len(data_preprocessed))+', '+str(len(data_preprocessed_labels))+'\n')
  f.write('(labels,count): '+str(list(zip(*np.unique(data_preprocessed_labels, return_counts=True)))))
  len_docs = [len(d.split(" ")) for d in data_preprocessed]
  f.write('\n min,mean,max docs len: '+str(np.min(len_docs))+', '+str(np.mean(len_docs).round(2))+', '+str(np.max(len_docs))+'\n\n')
  save_obj(data_preprocessed,'data_preprocessed'+'_'+data_to_get+'_'+dtype)
  save_obj(data_preprocessed_labels,'data_preprocessed_labels'+'_'+data_to_get+'_'+dtype)
  save_obj(vocab,'vocab'+'_'+data_to_get+'_'+dtype)
  save_obj(embeddings,'embeddings'+'_'+data_to_get+'_'+dtype)
  save_obj(df,'dataframe_'+data_to_get+'_'+dtype)

  ## skipgram embeddings short text
  all_tokens = [word_tokenize(t) for t in data_preprocessed]
  model = gensim.models.Word2Vec(all_tokens, min_count=0, sg=1, size=300,iter=200, workers=-1)
  generated_vocab = list(model.wv.vocab)
  generated_embeddings = {}
  for v in list(model.wv.vocab):
    vec = model.wv.__getitem__(v)
    generated_embeddings[v] =vec
  save_obj(generated_embeddings,'generated_embeddings_'+data_to_get+"_"+dtype)

  f.write('**'*50)

In [ ]:
#@title for sentences
# min_doc_len = 1
# min_word_len = 3
# import gensim
# split_to_sentences = True
# docs = df['docs'].values
# labels = df['labels'].values
# if split_to_sentences:
#   docs_sentences = [sent_tokenize(d) for d in docs]
#   allsent_docs = []
#   allsent_labels = []
#   for i in range(len(docs_sentences)):
#     # print(i)
#     data_preprocessed_sents,data_preprocessed_labels_sents,google_embeddings,vocab = docs_labels_preprocessing_sent(
#           docs_sentences[i],len(docs_sentences[i])*[labels[i]],word2vec_model,min_doc_len
#           ,min_word_len,max_features,split_to_sentences)


#     allsent_docs.append(data_preprocessed_sents)
#     allsent_labels.append(data_preprocessed_labels_sents)
#   allsent_docs = list(filter(None, allsent_docs))
#   allsent_labels = list(filter(None, allsent_labels))

#   df_sent = pd.DataFrame()
#   df_sent['allsent_docs'] = allsent_docs
#   df_sent['allsent_labels'] = allsent_labels

#   df_sent['allsent_docs_joined'] = df_sent['allsent_docs'].apply(lambda x: ' '.join(l for l in x).strip())
#   df_sent['allsent_labels_joined'] = df_sent['allsent_labels'].apply(lambda x: ' '.join(l for l in x).strip())
#   df_sent = df_sent.drop_duplicates(subset=['allsent_docs_joined'], keep='first').reset_index(drop=True)

#   allsent_docs = df_sent['allsent_docs'].values
#   allsent_labels = df_sent['allsent_labels'].values
#   all_sents = df_sent['allsent_docs_joined'].values

#   # all_sents = flatten(allsent_docs)
#   all_docs_sent_tokens = [word_tokenize(t) for t in all_sents]
#   model = gensim.models.Word2Vec(all_docs_sent_tokens, min_count=0, sg=1, size=300,iter=200,window=5, workers=-1)
#   generated_vocab = list(model.wv.vocab)
#   save_obj(df_sent,'sents_dataframe'+data_to_get+"_"+dtype)
#   save_obj(all_sents,'sents_allsents'+data_to_get+"_"+dtype)
#   save_obj(allsent_docs,'sents_allsent_docs_'+data_to_get+"_"+dtype)
#   save_obj(allsent_labels,'sents_allsent_labels_'+data_to_get+"_"+dtype)
#   generated_embeddings = {}
#   for v in list(model.wv.vocab):
#     vec = model.wv.__getitem__(v)
#     generated_embeddings[v] =vec

#   save_obj(google_embeddings,'sents_google_embeddings_'+data_to_get+"_"+dtype)
#   save_obj(vocab,'sents_countVec_vocab_'+data_to_get+"_"+dtype)
#   save_obj(generated_embeddings,'sents_generated_embeddings_'+data_to_get+"_"+dtype)
#   save_obj(generated_vocab,'sents_sg_generated_vocab_'+data_to_get+"_"+dtype)

In [ ]:
# import torch
# def DESM_score_Corpus(query_list, train_vec, vocab, embeddings):
#   num_docs = train_vec.shape[0]
#   num_voc = len(vocab)
#   sim_list = torch.zeros(num_docs)
#   index = 0
#   id_vocab = dict(map(reversed, vocab.items()))
#   for d in range(num_docs):
#     # if(d%5000==0): print(d)
#     doc_bar = torch.zeros(300)
#     doc_length = 0
#     for v in range(num_voc):
#       if(train_vec[d][v] > 0):
#         doc_bar.add_(train_vec[d][v] * torch.from_numpy(embeddings[id_vocab[v]])/torch.norm(torch.from_numpy(embeddings[id_vocab[v]])))
#         doc_length = doc_length + train_vec[d][v]
#     doc_bar = doc_bar / doc_length
#     sum = 0

#     for q in query_list:
#       sum += torch.dot(torch.from_numpy(embeddings[q]) , doc_bar)/(torch.norm(torch.from_numpy(embeddings[q]))*torch.norm(doc_bar))
#     sum = sum/len(query_list)
#     sim_list[index]=sum
#     index = index + 1

#   return sim_list

# # keywords = ['']
# # desm_score = DESM_score_Corpus(keywords, train_vec, vocab, embeddings)
# # sorted_desm_idx = torch.sort(desm_score,descending=True).indices

In [ ]:
d_data = 'yahooanswers'
import numpy as np

import torch.nn as nn
def get_embedding_tensor(word_list,embeddings): return torch.tensor([embeddings[w] for w in word_list])

cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-8)


def get_embedding_tensor(word_list,embeddings): return torch.tensor([embeddings[w] for w in word_list])

def get_all_keywords_score(keywords,word_list,embeddings):
    all_cosine_sim = []
    for k in keywords:
      keyword_embedding = torch.tensor(embeddings[k])
      word_embedding = get_embedding_tensor(word_list,embeddings)
      keyword_embedding = keyword_embedding.unsqueeze(0).expand(word_embedding.shape)
      cosine_sim_score = cos_sim(keyword_embedding,word_embedding)
      all_cosine_sim.append(cosine_sim_score)
    all_keywords_score = torch.stack(all_cosine_sim)
    return all_keywords_score


def flatten_list(user_list): return [item for sublist in user_list for item in sublist]
def toT(a): return torch.tensor(a)

def list_of_tensors_to_tensor(loT):
  stacked_tensor = torch.stack(loT)
  return stacked_tensor

def get_extended_keywords_list(new_data,keywords,embeddings,avg_doc_len):
  vec= CountVectorizer(min_df=0,dtype=np.uint8)
  vecdata = vec.fit_transform(new_data).toarray()
  vec_vocab = vec.vocabulary_

  # keywords = query
  word_list = sorted(vec_vocab)
  all_keywords_score = get_all_keywords_score(keywords,word_list,embeddings)
  most_similar_word_dict = {}
  extended_keywords_list = []
  # extend_each_by = int(avg_doc_len[d_data])
  extend_each_by  = 10
  for k in range(len(keywords)):
    v, i = torch.sort(all_keywords_score[k],descending=True)
    most_similar = np.array(word_list)[np.array(i)][:extend_each_by]
    extended_keywords_list.append(most_similar)
    most_similar_word_dict[keywords[k]] = most_similar
  return extend_each_by,extended_keywords_list


import torch
def DESM_score_Corpus2(query_list, train_vec, vocab, embeddings):
  sim_list = torch.zeros(train_vec.shape[0])
  index = 0
  word_list = np.asarray(sorted(vocab))
  for d in train_vec:
    words_in_d = np.where(d>0)[0]
    all_words_tensor = get_embedding_tensor(np.repeat(word_list[words_in_d],d[words_in_d]),embeddings)
    doc_bar = (all_words_tensor/torch.norm(all_words_tensor,dim=1).unsqueeze(-1)).sum(0) / d.sum(0)
    D_bar = doc_bar.unsqueeze(0).expand(len(query_list),doc_bar.shape[0])
    q = get_embedding_tensor(query_list,embeddings)
    norm_div = torch.norm(q,dim=1) * torch.norm(D_bar,dim=1)
    sim_list[index]=(torch.mm(q,D_bar.T)[:,0]/norm_div).sum()/len(query_list)
    index +=1
  return sim_list

avg_doc_len = {'bbc':13.8,
              'searchsnippet':13.16,
              'yahooanswers':11,
              'nfcorpus':7.72,
              'opinions_twitter':8.69}

# for d in set(data_preprocessed_labels):
#   embeddings[d]

In [ ]:
for s in set(data_preprocessed_labels):
  print(s.split('&'))

['tweets']


In [ ]:
from sklearn.preprocessing import MinMaxScaler
vectorizer = CountVectorizer(min_df=0,dtype=np.uint8)
train_vec = vectorizer.fit_transform(data_preprocessed).toarray()
keywords = ['business','finance','science','mathematics','society','culture','entertainment','music','sport','health','computer','internet','education','reference']
# get_extended_keywords_list(data_preprocessed,keywords,embeddings,avg_doc_len)

In [ ]:
for i in range(len(keywords)):
  for v in get_extended_keywords_list(data_preprocessed,keywords,embeddings,avg_doc_len)[1][i]:
    desm_score = DESM_score_Corpus2([v],train_vec,vocab,embeddings)
    scaler = MinMaxScaler()
    norm_desm_score = scaler.fit_transform(desm_score.reshape(-1,1)).reshape(-1)
    threshold = 0.6
    print(v,len(np.where(norm_desm_score > threshold)[0]))

In [ ]:
desm_score = DESM_score_Corpus2(['industrial'],train_vec,vocab,embeddings)
scaler = MinMaxScaler()
norm_desm_score = scaler.fit_transform(desm_score.reshape(-1,1)).reshape(-1)
threshold = 0.6
print(len(np.where(norm_desm_score > threshold)[0]))

146


In [ ]:
desm_score = DESM_score_Corpus(['engineering', 'engineer', 'design'],train_vec,vocab,embeddings)

In [ ]:
desm_score

tensor([0.2754, 0.4099, 0.3838,  ..., 0.1152, 0.0825, 0.1188])

In [ ]:
desm_score = DESM_score_Corpus2(['engineering', 'engineer', 'design'],train_vec,vocab,embeddings)

In [ ]:
desm_score

tensor([0.2754, 0.4099, 0.3838,  ..., 0.1152, 0.0825, 0.1188])

In [ ]:
# keywords = list(set(data_preprocessed_labels))
# keywords = ['engineering','sport','computer','business','health']
keywords = ['business','finance','science','mathematics','society','culture','entertainment','music','sport','health','computer','internet','education','reference']
get_extended_keywords_list(data_preprocessed,keywords,embeddings,avg_doc_len)

In [ ]:
# # # (popular, rare)
# # bbc_queries = [['film','piracy'], # (rare, rare)  (68,11) # entertainment
# #                ['chat','celebrity'], # (popular, rare)  (336,13) # entertainment
# #                ['hollywood','industry'] # (popular, popular)  (159,136) # entertainment
# #                ,
# #                ['private','security'],  # (popular, rare) (242,40) # tech
# #                ['corporate','sector'],  # (popular, popular) (247,266) # tech
# #                ['digital','art'] # (rare, rare) (53,9) # tech
# #                ,
# #                  ['bank','mortgage'], # (rare, rare) (33,42) # business
# #                  ['private','property'], # (popular, rare) (242,19) # business
# #                  ['global','operation'] # (popular, popular) (252,126) # business
# #                  ,
# #                  ['young','talent'], # (rare, popular)(41,122) # sport
# #                  ['race','sprinter'], # (rare, rare)(13 ,14) # sport
# #                  ['olympics','champion'] # (popular, popular)(336,155) # sport
# #                  ,
# #                  ['immigration','law'], # (rare,rare) (14,25) # politics
# #                  ['civil','administration'], # (rare,popular) (50,271) # politics
# #                  ['government','administration'] # (popular,popular) (195,271) # policy (policy-119)
# #                  ]


# ['sector','corporate'] # pp
# ['law','immigration'] # rr
# ['private','bank','debt'] # prr
# ['administration','policy','public'] # ppr

['administration', 'policy', 'public']

In [ ]:
from collections import Counter
allwords_inCorpus = []
for d in data_preprocessed:
  allwords_inCorpus.extend(word_tokenize(d))
allword_Count = Counter(allwords_inCorpus)
sorted(allword_Count)
allword_Count = dict(sorted(allword_Count.items(), key=lambda item: item[1], reverse=True))


In [ ]:
# os.chdir(home_dir)
# # data_to_get = 'yahooanswers'
# # max_features = 'all_vocab'
# zipped_pickle_filename = data_to_get+"_"+dtype
# os.system('zip -r '+zipped_pickle_filename+'_'+str(max_features)+'_.zip '+'./content/data_'+data_to_get)
# # files.download(zipped_pickle_filename+'_'+str(max_features)+'_.zip')

In [ ]:
STOP!!

In [ ]:
os.chdir(home_dir)

In [ ]:
# queryset = 1
# frac = 0.05
# os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype+'/injected_docs/'+str(frac))
# files = os.listdir('.')
# data_dict = decompress_pickle(files[queryset-1].split('.')[0])
# data_preprocessed = data_dict['new_docs']
# data_preprocessed_labels = data_dict['new_labels']
# ground_truth_docs = data_dict['docs_injected']
# ground_truth_labels = data_dict['labels_injected']
# keywords = data_dict['query']
# extended_keywords_list = data_dict['extended_keywords_list']

In [ ]:
# data =data_preprocessed
# labels = data_preprocessed_labels
# data = list(data)
# print('Before Injection : ',len(data),len(labels))

# keywords_as_labels = []
# extended_keywords_list_joined = np.asarray([list(extended_keywords_list[x]) + (list(extended_keywords_list[x+1])) for x in range(len(extended_keywords_list)) if x%2==0])
# keywords_as_labels = np.asarray(['p{'+keywords[x]+'&'+keywords[x+1]+'}' for x in range(len(keywords)) if x%2==0] )
# for e in range(len(extended_keywords_list_joined)):
#   data.append(' '.join(extended_keywords_list_joined[e]))
#   labels = np.append(labels,keywords_as_labels[e])
# print('After Injection : ',len(data),len(labels))

Before Injection :  37238 37238
After Injection :  37240 37240


## injection / query selection

In [ ]:
d_data = 'opinions_twitter'
dtype = 'short'
os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
data_preprocessed = load_obj_pkl5('data_preprocessed_'+d_data+'_'+dtype)
data_preprocessed_labels = load_obj_pkl5('data_preprocessed_labels_'+d_data+'_'+dtype)
df = load_obj_pkl5('dataframe_'+d_data+'_'+dtype)
embeddings = load_obj_pkl5('embeddings_'+d_data+'_'+dtype)
vocab = load_obj_pkl5('vocab_'+d_data+'_'+dtype)
queries_data_dict = decompress_pickle('queries_'+d_data)

In [ ]:
queries_data_dict

In [ ]:
len(data_preprocessed),len(embeddings)

(2259, 2000)

In [ ]:
#@title ICDM 2022 query and injecton class selection

# from collections import Counter
# if d_data !='nfcorpus' or d_data !='opinions_twitter':
#   d = Counter(data_preprocessed_labels)
#   asc_sorted_classes = sorted(d,key=d.get)
#   least_2_classes = asc_sorted_classes[:2]
#   least_3_classes = asc_sorted_classes[:3]
#   d,least_2_classes,least_3_classes
# print(least_2_classes,least_3_classes)


### bbc

# query = ['technology','entertainment']
# injection_class = least_2_classes

# query = ['technology','entertainment','political']
# injection_class = least_3_classes

### searchsnippet

# query = ['engineering','health']
# injection_class = least_2_classes

# query = ['engineering','health','sport']
# injection_class = least_3_classes

### yahooanswers
# query = ['science','mathematics','business','finance']
# injection_class = least_2_classes

# query = ['science','mathematics','business','finance','education','reference']
# injection_class = least_3_classes

### nfcorpus
# query = ['dietary', 'intervention', 'woman']
# injection_class = 'what does the research say about dietary interventions on women with pcos ?'

# query = ['coffee', 'mortality']
# injection_class = 'coffee and mortality'

### opinions_twitter
# query = ['job','hunting']
# injection_class = 'job hunting'

# query = ['kinect']
# injection_class = 'kinect'

In [ ]:
# from collections import Counter
# allwords_inCorpus = []
# for d in data_preprocessed:
#   allwords_inCorpus.extend(word_tokenize(d))
# allword_Count = Counter(allwords_inCorpus)
# sorted(allword_Count)
# allword_Count = dict(sorted(allword_Count.items(), key=lambda item: item[1], reverse=True))

# words= list(vocab.keys())
# import random
# queries = []
# for i in range(5):
#   queries.append(random.choices(words,k=2))

bbc_queries = [
['sector','corporate'],# pp
['law','immigration'], # rr
['administration','policy','public'], # ppr
['private','bank','debt']] # prr

searchsnippet_queries = [
['biz','economics'], #  pp (1437,612)
['healthcare','fitness'], # rr (48,36)
['sporting','athlete','racing'], # ppr (780,690,63)
['notebook','microprocessor','disk']] # prr (549,79,46)

yahooanswers_queries = [
['lifestyle','school'], #pp
['disease','diabetes'], # rr
['musical','song','guitar'], # ppr
['youtube','social','gaming'] #prr
]

#

#['treating','dry','eye','disease','diet','water'],
#['living', 'longer', 'reducing', 'leucine', 'intake'],
#['rye','bread','protect','cancer'],
#['fish' ,'consumption', 'suicide']
#]
#

nfcorpus_model_queries = [
['dietary', 'intervention', 'woman'],
['coffee','mortality'],
['bowel','movement'],
['diabetes'],
['eye','disease'],
['living','longer','leucine'],
['rye','bread','cancer'],
['fish' ,'consumption', 'suicide']
]


nfcorpus_whole_queries = ['what does the research say about dietary interventions on women with pcos ?',
                          'coffee and mortality',
                          'how many bowel movements should you have every day ?',
                          'what causes diabetes ?',
                          'treating dry eye disease with diet : just add water ?',
                          'living longer by reducing leucine intake',
                          'does rye bread protect against cancer ?',
                          'fish consumption and suicide']

opinions_twitter_queries = [
['job','hunting'],
['kinect'],
['htc'],
['speech','recognition'],
['iran','nuclear'],
['obama'],
['manchester'],
['pixar']]

opinions_twitter_whole_queries = ['job hunting','kinect','htc','speech recognition','iran nuclear','Obama','manchester city','pixar']

if d_data =='bbc': queries = bbc_queries
elif d_data =='searchsnippet': queries = searchsnippet_queries
elif d_data =='yahooanswers': queries = yahooanswers_queries
elif d_data =='nfcorpus': queries = nfcorpus_model_queries
elif d_data =='opinions_twitter': queries = opinions_twitter_queries
else: queries = []

avg_doc_len = {'bbc':12.01,
              'searchsnippet':13.16,
              'yahooanswers':11,
              'nfcorpus':7.72,
              'opinions_twitter':7.15
              #  'opinions_twitter':12
               }

# avg_doc_len = {'bbc':3,
#               'searchsnippet':3,
#               'yahooanswers':3,
#               'nfcorpus':3,
#                'opinions_twitter':3}

In [ ]:
# ss = (embeddings['bowel'] + embeddings['movement'])

In [ ]:
# job + hunting: ['hunting', 'job', 'work', 'employment', 'doe', 'turkey', 'muskrat']
# speech + recognition: ['speech', 'recognition', 'presentation', 'lecture', 'keynote','award', 'message']
# bowel + movement: ['bowel', 'movement', 'bladder', 'intestine', 'colon', 'gastric','esophagus']

In [ ]:
for query in queries:
  for q in query: embeddings[q]

In [ ]:
from collections import defaultdict
import torch.nn as nn
import numpy as np
import torch
import random

seed = 2022
random.seed(seed)
np.random.seed(seed=seed)


def get_docs_idx_in_vis(relv_docs,preprossed_data_non_zeros,doc_ids):
  d = {item: idx for idx, item in enumerate(preprossed_data_non_zeros)}
  doc_ids_list = list(doc_ids)
  relv_docs_idx = [d.get(item) for item in relv_docs]
  docs_in_vis_idx = [doc_ids_list.index(r_i) for r_i in relv_docs_idx]
  assert (doc_ids[docs_in_vis_idx] == relv_docs_idx).all() == True
  return docs_in_vis_idx

def vocab_filtered_data(doc,vocab):
  doc = word_tokenize(doc)
  doc = filter(lambda x: x in vocab, doc)
  doc = ' '.join(e for e in doc)
  return doc

### IR Datasets
# df['docs_preprocessed_vocab_filtered'] = df['docs_preprocessed'].apply(lambda x: vocab_filtered_data(x,vocab))
# df = df[df['docs_preprocessed_vocab_filtered'].isin(data_preprocessed)].reset_index(drop=True)
# ground_truth_docs = df[df['queries']==injection_class[0]]['docs_preprocessed_vocab_filtered'].values

# d = {item: idx for idx, item in enumerate(data_preprocessed)}
# relv_docs_idx = [d.get(item) for item in ground_truth_docs]

def injection(query,injection_class,frac,data_preprocessed,data_preprocessed_labels):
  all_injected_docs = []
  all_injected_labels = []
  all_txt_inj = []
  d = Counter(data_preprocessed_labels)
  # injection_class = sorted(d, key=d.get)[:2]
  # injection_class = ['Sports','Health']
  np_data = np.asarray(data_preprocessed)
  np_labels = np.asarray(data_preprocessed_labels)

  inject_to =  ~np.isin(np_labels, injection_class)
  inject_to_docs = np_data[inject_to]
  inject_to_labels = np_labels[inject_to]

  p=frac

  for i in range(len(injection_class)):
    to_inject= np.where(np_labels == injection_class[i])[0]
    rand_perm_k_to_inject = np.random.RandomState(seed=seed).permutation(to_inject)
    to_inject_docs = np_data[rand_perm_k_to_inject]
    take_docs_to_inject = to_inject_docs[:round(len(to_inject_docs)*p)]
    all_txt_inj.append((str(frac)+" "+str(len(take_docs_to_inject))+' docs injected of '+injection_class[i]))
    all_injected_docs.append(take_docs_to_inject)
    take_labels_to_inject = len(take_docs_to_inject)*['injected_'+injection_class[i]]
    all_injected_labels.append(take_labels_to_inject)
    inject_to_docs = np.asarray(list(inject_to_docs) + list(take_docs_to_inject))
    inject_to_labels = np.asarray(list(inject_to_labels) + list(take_labels_to_inject))

  # sorted_unique_labels = sorted(set(inject_to_labels))
  print(all_txt_inj)
  new_data = inject_to_docs
  new_labels = inject_to_labels
  assert  len(new_data) == len(new_labels)
  return new_data,new_labels,all_injected_docs,all_injected_labels,all_txt_inj

cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-8)

def get_embedding_tensor(word_list,embeddings): return torch.tensor([embeddings[w] for w in word_list])

def get_all_keywords_score(keywords,word_list,embeddings):
    all_cosine_sim = []
    for k in keywords:
      keyword_embedding = torch.tensor(embeddings[k])
      word_embedding = get_embedding_tensor(word_list,embeddings)
      keyword_embedding = keyword_embedding.unsqueeze(0).expand(word_embedding.shape)
      cosine_sim_score = cos_sim(keyword_embedding,word_embedding)
      all_cosine_sim.append(cosine_sim_score)
    all_keywords_score = torch.stack(all_cosine_sim)
    return all_keywords_score

def get_extended_keywords_list(new_data,keywords,embeddings,avg_doc_len,th,use_th):
  vec= CountVectorizer(min_df=0,dtype=np.uint8)
  vecdata = vec.fit_transform(new_data).toarray()
  vec_vocab = vec.vocabulary_

  word_list = sorted(vec_vocab)
  all_keywords_score = get_all_keywords_score(keywords,word_list,embeddings)
  most_similar_word_dict = {}
  extended_keywords_list = []
  extend_each_by = int(avg_doc_len[d_data])

  for k in range(len(keywords)):
    v, i = torch.sort(all_keywords_score[k],descending=True)
    if not use_th:
      most_similar = np.array(word_list)[np.array(i)][:extend_each_by]
    else:
      th_idx = i[torch.where(v>=th)[0]]
      most_similar = np.array(word_list)[np.array(th_idx)]
    extended_keywords_list.append(most_similar)
    most_similar_word_dict[keywords[k]] = most_similar
  return extend_each_by,extended_keywords_list

# def f7(seq):
#     seen = set()
#     seen_add = seen.add
#     return [x for x in seq if not (x in seen or seen_add(x))]
# def split_list(a_list):
#     half = len(a_list)//2
#     return a_list[:half], a_list[half:]

In [ ]:
if d_data == 'bbc' or d_data == 'searchsnippet' or  d_data == 'yahooanswers':
  extended_keywords_list_allQ = []
  extend_each_by_allQ = []
  query_dict = {}
  for q in range(len(queries)):
    extend_each_by,extended_keywords_list = get_extended_keywords_list(data_preprocessed,queries[q],embeddings,avg_doc_len,th,use_th=False)
    # extended_keywords_list_allQ.append(extended_keywords_list)
    # extend_each_by_allQ.append(extend_each_by)

    data_dict = {}
    data_dict['extend_each_by']  = extend_each_by
    data_dict['extended_keywords_list'] = extended_keywords_list
    data_dict['query'] = queries[q]
    data_dict['whole_query'] = queries[q]
    query_dict[str(q+1)] = data_dict

  os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype,exist_ok=True)
  os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
  compressed_pickle(query_dict,'queries_'+d_data)

In [ ]:
if d_data == 'nfcorpus' or d_data == 'opinions_twitter':
  extended_keywords_list_allQ = []
  extend_each_by_allQ = []
  th = 0.0
  query_dict = {}
  for q in range(len(queries)):
    extend_each_by,extended_keywords_list = get_extended_keywords_list(data_preprocessed,queries[q],embeddings,avg_doc_len,th,use_th=False)

    data_dict = {}
    data_dict['extend_each_by']  = extend_each_by
    data_dict['extended_keywords_list'] = extended_keywords_list
    data_dict['query'] = queries[q]
    # data_dict['whole_query'] = queries[q]

    if d_data == 'nfcorpus' or d_data == 'opinions_twitter':

      df['docs_preprocessed_vocab_filtered'] = df['docs_preprocessed'].apply(lambda x: vocab_filtered_data(x,vocab) )
      df = df[df['docs_preprocessed_vocab_filtered'].isin(data_preprocessed)].reset_index(drop=True)

      if d_data == 'nfcorpus':
        df_all_qdocs = df[df['queries']==nfcorpus_whole_queries[q]].reset_index(drop=True)
        ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']==2]['docs_preprocessed_vocab_filtered'].values
        ground_truth_labels = 'MED'+df_all_qdocs[df_all_qdocs['relevance']==2]['relevance'].astype('str').values

      elif d_data == 'opinions_twitter':
        df_all_qdocs = df[df['queries']==opinions_twitter_whole_queries[q]].reset_index(drop=True)
        ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']=='T']['docs_preprocessed_vocab_filtered'].values
        ground_truth_labels = df_all_qdocs[df_all_qdocs['relevance']=='T']['relevance'].values

      data_dict['ground_truth_docs'] = ground_truth_docs
      data_dict['ground_truth_labels'] = ground_truth_labels
      # data_dict['df_all_qdocs'] = df_all_qdocs

    query_dict[str(q+1)] = data_dict

  os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype,exist_ok=True)
  os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
  compressed_pickle(query_dict,'queries_'+d_data)

In [ ]:
qs = '4'

In [ ]:
keywords = query_dict[qs]['query']
whole_query = [' '.join(keywords)]
vec= CountVectorizer(min_df=0,dtype=np.uint8)
vecdata = vec.fit_transform(data_preprocessed).toarray()
vec_vocab = vec.vocabulary_

word_list = sorted(vec_vocab)

all_cosine_sim = []
for k in keywords:
  keyword_embedding = torch.tensor(embeddings[k])
  word_embedding = get_embedding_tensor(word_list,embeddings)
  keyword_embedding = keyword_embedding.unsqueeze(0).expand(word_embedding.shape)
  cosine_sim_score = cos_sim(keyword_embedding,word_embedding)
  all_cosine_sim.append(cosine_sim_score)

most_similar_word_dict = {}
extended_keywords_list_together = []
extend_each_by = 6

cosine_sim_score = list_of_tensors_to_tensor(all_cosine_sim).sum(0)
# # for k in range(len(keywords)):
v, i = torch.sort(cosine_sim_score,descending=True)
# if not use_th:
most_similar = np.array(word_list)[np.array(i)][:extend_each_by]
#   else:
#     th_idx = i[torch.where(v>=th)[0]]
#     most_similar = np.array(word_list)[np.array(th_idx)]
extended_keywords_list_together.append(most_similar)
# most_similar_word_dict[keywords[k]] = most_similar

In [ ]:
extended_keywords_list_together,whole_query

([array(['recognition', 'speech', 'presentation', 'award', 'lecture',
         'keynote'], dtype='<U14')],
 ['speech recognition'])

In [ ]:
query_dict[qs]['extended_keywords_list'] = extended_keywords_list_together
query_dict[qs]['whole_query'] = whole_query
query_dict[qs]['extend_each_by'] = extend_each_by
compressed_pickle(query_dict,'queries_'+d_data)

In [ ]:
query_dict

In [ ]:
# extended_keywords_list_allQ = []
# extend_each_by_allQ = []
# th = 0.0
# query_dict = {}
# for q in range(len(queries)):
#   extend_each_by,extended_keywords_list = get_extended_keywords_list(data_preprocessed,queries[q],embeddings,avg_doc_len,th,use_th=True)

#   data_dict = {}
#   data_dict['extend_each_by']  = extend_each_by
#   data_dict['extended_keywords_list'] = extended_keywords_list
#   data_dict['query'] = queries[q]
#   query_dict[str(q+1)] = data_dict

# os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype,exist_ok=True)
# os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
# compressed_pickle(query_dict,'queries_'+d_data+'_'+str(th))

In [ ]:
if d_data == 'opinions_twitter' or d_data == 'nfcorpus':
  frac = 0.2
  data_dict = {}
  df['docs_preprocessed_vocab_filtered'] = df['docs_preprocessed'].apply(lambda x: vocab_filtered_data(x,vocab) )
  df = df[df['docs_preprocessed_vocab_filtered'].isin(data_preprocessed)].reset_index(drop=True)

  df_all_qdocs = df[df['queries']==injection_class].reset_index(drop=True)
  opinions_twitter_ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']=='T']['docs_preprocessed_vocab_filtered'].values
  m
  df_all_qdocs = df[df['queries']==injection_class].reset_index(drop=True)
  nfcorpus_ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']==2]['docs_preprocessed_vocab_filtered'].values


  # ground_truth_docs = df[df['queries']==injection_class[0]]['docs_preprocessed'].values
  nfcorpus_ground_truth_labels = 'MED'+df_all_qdocs[df_all_qdocs['relevance']==2]['relevance'].astype('str').values
  opinions_twitter_ground_truth_labels = df_all_qdocs[df_all_qdocs['relevance']=='T']['relevance'].values

  extend_each_by,extended_keywords_list = get_extended_keywords_list(data_preprocessed,query,query,embeddings,avg_doc_len)
  data_dict['new_docs'] = df['docs_preprocessed_vocab_filtered'].tolist()
  data_dict['new_labels'] = df['labels'].tolist()


  if d_data == 'opinions_twitter':
    data_dict['docs_injected'] = opinions_twitter_ground_truth_docs
    data_dict['labels_injected'] = opinions_twitter_ground_truth_labels
    data_dict['info']  = str(len(opinions_twitter_ground_truth_docs))+' relevant docs for query '+str(query)

  elif d_data == 'nfcorpus':
    data_dict['docs_injected'] = nfcorpus_ground_truth_docs
    data_dict['labels_injected'] = nfcorpus_ground_truth_labels
    data_dict['info']  = str(len(nfcorpus_ground_truth_docs))+' directly relevant docs for query '+str(query)

  data_dict['injection_class'] = injection_class
  data_dict['extend_each_by']  = extend_each_by
  data_dict['extended_keywords_list'] = extended_keywords_list
  data_dict['query'] = query

  os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac),exist_ok=True)

  os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac))
  compressed_pickle(data_dict,'injected_'+d_data+"_"+'_'.join(query)+"_injClass_"+'_'.join(injection_class.split(' ')))

In [ ]:
  all_aspects_all_keywords = flatten_list(extended_keywords_list)
  query_as_doc = ' '.join(all_aspects_all_keywords)

  #### DESM
  desm_score = DESM_score_Corpus(all_aspects_all_keywords, train_vec, vocab, embeddings)
  sorted_desm_idx = torch.sort(desm_score,descending=True).indices
  ####

  #### TF-IDF
  tfdifvec = TfidfVectorizer()
  tfdifvec.fit(preprossed_data_non_zeros)
  tfdif_doc_vectors = torch.from_numpy(tfdifvec.transform(preprossed_data_non_zeros).toarray())
  tfdif_query_vectors = torch.from_numpy(tfdifvec.transform([query_as_doc]).toarray())

  tfidf_score = cos_sim(tfdif_query_vectors,tfdif_doc_vectors)
  sorted_tfidf_idx = torch.sort(tfidf_score,descending=True).indices


In [ ]:
data_to_get = 'searchsnippet'
paper='_emnlp_2022'
os.chdir(home_dir)
# max_features = 'all_vocab'
zipped_pickle_filename = data_to_get+"_"+dtype+paper
os.system('zip -r '+zipped_pickle_filename+'_.zip '+'./content/data_'+data_to_get)
# files.download(zipped_pickle_filename+'_'+str(max_features)+'_.zip')

0

In [ ]:
%pwd

'/home/grad16/sakumar/ICDM_Experiments_2022/dataset'

In [ ]:
STOP!!

In [ ]:
#@title ICDM 2022 document injection
# avg_doc_len = {'bbc':13.8,
#               'searchsnippet':14.87,
#               'yahooanswers':11.82,
#               'nfcorpus':8.19,
#               'opinions_twitter':8.69}

# if d_data == 'opinions_twitter' or d_data == 'nfcorpus':
#   pass
# else:
#   fracs = [0.05,0.1,0.15,0.2]
#   # fracs = [0.15]
#   for frac in fracs:
#     new_data , new_labels, all_injected_docs,all_injected_labels,all_txt_inj = injection(query,injection_class,frac,data_preprocessed,data_preprocessed_labels)
#     extend_each_by,extended_keywords_list = get_extended_keywords_list(new_data,query,query,embeddings,avg_doc_len)

#     if d_data == "yahooanswers":
#       extended_keywords_list_o = extended_keywords_list[:]
#       extended_keywords_list_joined = np.asarray([list(extended_keywords_list_o[x]) + (list(extended_keywords_list_o[x+1])) for x in range(len(extended_keywords_list_o)) if x%2==0])

#       # extend_each_by = int(extend_each_by/2)
#       extended_keywords_list_oo =[]
#       for ex in range(len(extended_keywords_list_o)):
#         ext = extended_keywords_list_o[ex]
#         extended_keywords_list_oo0 = []
#         for e in range(len(ext)):
#           if e!=0 and ext[e] in extended_keywords_list_joined:
#             pass
#           if ex<len(extended_keywords_list_o)-1:
#             if ext[e] in extended_keywords_list_o[ex+1]:
#               pass
#             else:
#               extended_keywords_list_oo0.append(ext[e])
#         extended_keywords_list_oo0.extend(extended_keywords_list_o[ex][:int(extend_each_by/2)])
#         extended_keywords_list_oo.append(np.asarray(extended_keywords_list_oo0[:int(extend_each_by/2)]))
#         extended_keywords_list = extended_keywords_list_oo


#     data_dict = {}
#     data_dict['new_docs'] = new_data
#     data_dict['new_labels'] = new_labels
#     data_dict['docs_injected'] = all_injected_docs
#     data_dict['labels_injected'] = all_injected_labels
#     data_dict['injection_class'] = injection_class
#     data_dict['extend_each_by']  = extend_each_by
#     data_dict['extended_keywords_list'] = extended_keywords_list
#     data_dict['query'] = query
#     data_dict['info']  = all_txt_inj

#     os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac),exist_ok=True)
#     # os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac),exist_ok=True)

#     os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac))
#     compressed_pickle(data_dict,'injected_'+d_data+"_"+'_'.join(query)+"_injClass_"+'_'.join(injection_class))

In [ ]:
# extended_keywords_list_joined = np.asarray([list(extended_keywords_list_oo[x]) + (list(extended_keywords_list_oo[x+1])) for x in range(len(extended_keywords_list_oo)) if x%2==0])
# extended_keywords_list_joined = [np.asarray(e) for e in extended_keywords_list_joined]

### IR dataset

In [ ]:
# !rm -r /home/grad16/sakumar/ICDM_Experiments_2022/WTM/SavedOutput/

In [ ]:
if d_data == 'opinions_twitter' or d_data == 'nfcorpus':
  frac = 0.2
  data_dict = {}
  df['docs_preprocessed_vocab_filtered'] = df['docs_preprocessed'].apply(lambda x: vocab_filtered_data(x,vocab) )
  df = df[df['docs_preprocessed_vocab_filtered'].isin(data_preprocessed)].reset_index(drop=True)

  df_all_qdocs = df[df['queries']==injection_class].reset_index(drop=True)
  opinions_twitter_ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']=='T']['docs_preprocessed_vocab_filtered'].values

  df_all_qdocs = df[df['queries']==injection_class].reset_index(drop=True)
  nfcorpus_ground_truth_docs = df_all_qdocs[df_all_qdocs['relevance']==2]['docs_preprocessed_vocab_filtered'].values


  # ground_truth_docs = df[df['queries']==injection_class[0]]['docs_preprocessed'].values
  nfcorpus_ground_truth_labels = 'MED'+df_all_qdocs[df_all_qdocs['relevance']==2]['relevance'].astype('str').values
  opinions_twitter_ground_truth_labels = df_all_qdocs[df_all_qdocs['relevance']=='T']['relevance'].values

  extend_each_by,extended_keywords_list = get_extended_keywords_list(data_preprocessed,query,query,embeddings,avg_doc_len)
  data_dict['new_docs'] = df['docs_preprocessed_vocab_filtered'].tolist()
  data_dict['new_labels'] = df['labels'].tolist()


  if d_data == 'opinions_twitter':
    data_dict['docs_injected'] = opinions_twitter_ground_truth_docs
    data_dict['labels_injected'] = opinions_twitter_ground_truth_labels
    data_dict['info']  = str(len(opinions_twitter_ground_truth_docs))+' relevant docs for query '+str(query)

  elif d_data == 'nfcorpus':
    data_dict['docs_injected'] = nfcorpus_ground_truth_docs
    data_dict['labels_injected'] = nfcorpus_ground_truth_labels
    data_dict['info']  = str(len(nfcorpus_ground_truth_docs))+' directly relevant docs for query '+str(query)

  data_dict['injection_class'] = injection_class
  data_dict['extend_each_by']  = extend_each_by
  data_dict['extended_keywords_list'] = extended_keywords_list
  data_dict['query'] = query

  os.makedirs(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac),exist_ok=True)

  os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype+"/injected_docs/"+str(frac))
  compressed_pickle(data_dict,'injected_'+d_data+"_"+'_'.join(query)+"_injClass_"+'_'.join(injection_class.split(' ')))

In [ ]:
# from collections import Counter
Counter(data_preprocessed_labels)

In [ ]:
# files = os.listdir('.')
# d = decompress_pickle(files[0].split('.')[0])

In [ ]:
STOP!!

In [ ]:
#@title opinions_twitter docs (relv and irrelv)
opinions_twitter_iran_nuclear_relvdocs = np.array([['F',
        'iran worked nuclear bomb design iaea iran worked developing nuclear weapon design resear http'],
       ['F',
        'connecting nuclear dot iran connecting nuclear dot iran regard http'],
       ['F',
        'demand iran respond day nuclear report reuters demand iran respond day nuclear http'],
       ['F',
        'barak nuclear iran launch arm race usa nuclear iran launch arm video def'],
       ['F',
        'power close deal iaea iran resolution http attack iran anti iran scenario sanction nuclear program'],
       ['F',
        'forget stuxnet israel considering striking iran nuclear program nuclear weapon http'],
       ['T', 'israel iran nuclear weapon http state nuclear weapon'],
       ['F', 'imo iran develop nuclear weapon develop nuclear weapon'],
       ['F',
        'truth freed iran nuclear bomb zionist regime israel nuclear warhead threat'],
       ['F',
        'iaea claim iran developing nuclear bomb http respect resisted pressure claim iraq nuclear'],
       ['F',
        'time nuclear watchdog admits suspicion iran atomic energy program build nuclear weapon http'],
       ['F',
        'time nuclear watchdog admits suspicion iran atomic energy program build nuclear weapon http'],
       ['T',
        'problem iranian nuclear issue faith game nuclear chicken israel iran'],
       ['F',
        'iran nuclear program keep playing drama pakistan nuclear accepting http'],
       ['F',
        'iran nuclear defiance tap deep point pride cbs news nuclear defiance tap deep point http'],
       ['F',
        'iran nuclear site observed sign cleanup http globe nuclear site bei http'],
       ['T', 'don terrorist mullah iran nuclear bomb terrorism iran'],
       ['F',
        'israel strike iran speculation fed looming report iran nuclear activity http'],
       ['T',
        'iran missile nuclear site iran difficult managing price crude oil gaza'],
       ['F', 'report cite secret nuclear iran http digg iran'],
       ['F', 'report cite secret nuclear iran http digg iran'],
       ['F',
        'breakingnews iran president iran won retreat iota nuclear program'],
       ['F', 'iran build ethic nuclear bomb http iran'],
       ['F',
        'mike photographic proof iran nuclear laboratory http iran iaea nuke humor'],
       ['T',
        'iran nuclear report alarming white house http political iran'],
       ['T', 'elect barackobama iran nuclear weapon elect iran'],
       ['F',
        'domestic election iran news iran nuclear capability agreed stunt'],
       ['F', 'iran western nuclear sanction vain iran canada http'],
       ['F',
        'iran arrest cia spy accused targeting nuclear iran usa cia http'],
       ['F', 'word iran nuclear weapon shut strait oil barrel iran'],
       ['T',
        'harboring illusion iran peaceful intent nuclear power iran'],
       ['T', 'iran nuke claim credible iran nuclear program http'],
       ['F',
        'iran threat closely israel warning window opportunity iran nuclear closing http'],
       ['T',
        'israel warning iran quiet nod gulf http linked shared fear iran nuclear program'],
       ['F',
        'concern exist iran mullah easily obtaining nuclear weapon http iran terrorism hezbollah'],
       ['T',
        'russia issue stark warning attack iran iaea expected iran nearing nuclear capability http'],
       ['F',
        'iran working advanced warhead http publish detail advanced design iran nuclear warhead'],
       ['F',
        'iran worked nuclear bomb design watchdog vienna reuters iran appears work http'],
       ['F',
        'yahoo news iran worked nuclear bomb design news report iran worked http'],
       ['T',
        'iran nuclear chief american pawn iran president criticized head http'],
       ['T',
        'strong kay iran president iran won retreat iota nuclear program'],
       ['F',
        'serious concern iran nuke atomic agency serious concern iran nuclear'],
       ['T',
        'eng iran serious proof nuclear drive israel support crippling sanction iran http'],
       ['T',
        'iran hit nuclear weapon allegation president iran retreat http'],
       ['F',
        'report complicate iran defense nuclear program iran dismissed united nation report http'],
       ['F',
        'obama iran nuclear program main alternative military operation iran http'],
       ['T',
        'iran vow counter israeli strike iron fist metaphor survived silly iran iron age nuclear age'],
       ['T', 'obama iran failure http obama fail iran nuclear cluster'],
       ['F',
        'redding news demand iran respond day nuclear report reuters demand iran respond day http'],
       ['T',
        'elect barack obama iran nuclear weapon elect mitt romney iran http'],
       ['T',
        'romney war keep iran nuclear weapon http obama iran nuke ows dry'],
       ['F',
        'jonathan cook striking iran israel war wager http palestine iran nuclear occu'],
       ['T',
        'iran dismisses iaea report meaningless iran review operation nuclear watchdog http']],
      dtype=object)

### save

In [ ]:
os.chdir(home_dir)
d_data = 'yahooanswers'
dtype='short'
# data_to_get = 'yahooanswers'
# max_features = 'all_vocab'
zipped_pickle_filename = 'FoTo_'+d_data+"_"+dtype
os.system('zip -r '+zipped_pickle_filename+'.zip '+'./content/data_'+d_data)
# files.download(zipped_pickle_filename+'_'+str(max_features)+'_.zip')

## twitter pick queries

In [ ]:
d_data = 'opinions_twitter'
dtype = 'short'
os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
data_preprocessed = load_obj_pkl5('data_preprocessed_'+d_data+'_'+dtype)
data_preprocessed_labels = load_obj_pkl5('data_preprocessed_labels_'+d_data+'_'+dtype)
df = load_obj_pkl5('dataframe_'+d_data+'_'+dtype)
embeddings = load_obj_pkl5('embeddings_'+d_data+'_'+dtype)
vocab = load_obj_pkl5('vocab_'+d_data+'_'+dtype)

In [ ]:
embeddings['nuclear']

In [ ]:
[['iran','nuclear'],
['obama'],
['manchester'],
['pixar']]

# ['iran nuclear','Obama,'manchester city','pixar']

[['iran', 'nuclear'], ['obama'], ['manchester'], ['pixar']]

In [ ]:
df2 = df.groupby(['queries','relevance']).size().reset_index(name='counts')
df3 = df2[df2['relevance']=='T'].reset_index(drop=True)
df3.sort_values(by='counts',ascending=False)

,queries,relevance,counts
26,job hunting,T,28
28,kinect,T,22
2,Jennifer Aniston,T,21
1,Breaking Dawn,T,20
24,iran nuclear,T,19
15,chris paul,T,19
7,Obama,T,19
13,big bang,T,15
31,manchester city,T,15
19,htc,T,14


In [ ]:
top_directlinks_queries = df3.sort_values(by='counts',ascending=False)['queries'].values[:5]
top_directlinks_queries

array(['job hunting', 'kinect', 'Jennifer Aniston', 'Breaking Dawn',
       'iran nuclear'], dtype=object)

In [ ]:
query = ['job','hunting']
injection_class = 'job hunting'

query = ['kinect']
injection_class = 'kinect'

In [ ]:
# queryset = 1
# frac = 0.2
# os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype+'/injected_docs/'+str(frac))
# files = os.listdir('.')
# data_dict = decompress_pickle(files[queryset-1].split('.')[0])
# data_preprocessed = data_dict['new_docs']
# data_preprocessed_labels = data_dict['new_labels']
# ground_truth_docs = data_dict['docs_injected']
# ground_truth_labels = data_dict['labels_injected']
# keywords = data_dict['query']
# extended_keywords_list = data_dict['extended_keywords_list']

## nfcorpus pick queries

In [ ]:
os.chdir(home_dir)

In [ ]:
d_data = 'nfcorpus'
dtype = 'short'
os.chdir(home_dir+'/content/data_'+d_data+"/"+dtype)
data_preprocessed = load_obj_pkl5('data_preprocessed_'+d_data+'_'+dtype)
data_preprocessed_labels = load_obj_pkl5('data_preprocessed_labels_'+d_data+'_'+dtype)
df = load_obj_pkl5('dataframe_'+d_data+'_'+dtype)
embeddings = load_obj_pkl5('embeddings_'+d_data+'_'+dtype)
vocab = load_obj_pkl5('vocab_'+d_data+'_'+dtype)

In [ ]:
df2 = df.groupby(['queries','relevance']).size().reset_index(name='counts')

In [ ]:
df3 = df2[df2['relevance']==2].reset_index(drop=True)
df3.sort_values(by='counts',ascending=False)['queries'].values

In [ ]:
df3.sort_values(by='counts',ascending=False).head(20)

,queries,relevance,counts
111,what does the research say about dietary inter...,2,20
27,coffee and mortality,2,17
96,treating dry eye disease with diet : just add ...,2,15
53,how many bowel movements should you have every...,2,15
59,how to boost the benefits of exercise,2,13
109,what causes diabetes ?,2,11
79,living longer by reducing leucine intake,2,11
38,does rye bread protect against cancer ?,2,11
85,putrefying protein and “ toxifying ” enzymes,2,10
41,fish consumption and suicide,2,10


In [ ]:
top_directlinks_queries = df3.sort_values(by='counts',ascending=False)['queries'].values[:20]
top_directlinks_queries

array(['what does the research say about dietary interventions on women with pcos ?',
       'coffee and mortality',
       'treating dry eye disease with diet : just add water ?',
       'how many bowel movements should you have every day ?',
       'how to boost the benefits of exercise', 'what causes diabetes ?',
       'living longer by reducing leucine intake',
       'does rye bread protect against cancer ?',
       'putrefying protein and “ toxifying ” enzymes',
       'fish consumption and suicide',
       'brown fat : losing weight through thermogenesis',
       'should you sit , squat , or lean during a bowel movement ?',
       'why deep fried foods may cause cancer',
       'foods for macular degeneration',
       'flame retardant pollutants and child development',
       'how may plants protect against diabetes ?',
       'can we fight the blues with greens ?',
       'fatty meals may impair artery function',
       'does chocolate cause weight gain ?',
       'organic mil

In [ ]:
query_words = top_directlinks_queries[0].split(' ')
query_words_w_emb = []
for q in query_words:
  try:
    embeddings[q]
    query_words_w_emb.append(q)
  except:
    pass

In [ ]:
query_words #  dietary intervention woman

['what',
 'does',
 'the',
 'research',
 'say',
 'about',
 'dietary',
 'interventions',
 'on',
 'women',
 'with',
 'pcos',
 '?']

In [ ]:
# df[df['queries']=='what does the research say about dietary interventions on women with pcos ?']
# df[df['queries']=='coffee and mortality']

In [ ]:
query = ['dietary', 'intervention', 'woman']
injection_class = 'what does the research say about dietary interventions on women with pcos ?'

query = ['coffee','mortality']
injection_class = 'coffee and mortality'

### old queries


In [ ]:

### nfcorpus
# query = ['benefit','exercise']
# injection_class = ['how to boost the benefits of exercise']

# query = ['food','macular','degeneration']
# injection_class = ['foods for macular degeneration']

### opinions_twitter
# query = ['iran','nuclear']
# injection_class = ['iran nuclear']

# query = ['big','bang']
# injection_class = ['big bang']